## Primer: Working with Arrays using Numpy  

"Nearly every scientist working in Python draws on the power of Numpy."  
[Numpy.org](https://numpy.org/)  

Numpy is widely used across scientific disciplines for numerical computing, and is the most common means of working with data arrays.   


```python
import numpy as np
```

Numpy arrays can have multiple dimensions.  

To create a one dimensional array from scratch, use ```np.array()``` with a list of numbers (integers or floats) as the sole parameter:

```python
np.array([4,6,4,7,2])
```

Usually, though an array is two-dimensional.  

To create a 2D array from scratch, you need nested lists:  

```python
np.array([[4,6,4,7,2],[5,8,4,9,7],[6,8,3,4,2],[9,5,8,6,7],[5,2,7,9,1],[7,4,2,6,7]])
```

In [ ]:
np.array([[4,6,4,7,2],[5,8,4,9,7],[6,8,3,4,2],[9,5,8,6,7],[5,2,7,9,1],[7,4,2,6,7]])

So what is an array? You can think of it as a grid. 

What types of data that you might work with are gridded?  

Store the above array into a variable:  
```python
arr = np.array(([4,6,4,7,2],[5,8,4,9,7],[6,8,3,4,2],[9,5,8,6,7],[5,2,7,9,1],[7,4,2,6,7]))
```

Often, you need to know the shape of your array, particularly for transforming it to different data formats such as netCDF or GeoTiff.

```python
arr.shape
```

Note that shape is in (rows, columns) notation.  

Use the ```.size``` method to get the size: 
```python
arr.size
```

Use the ```.ndim``` method to get it's dimensions.

Indexing arrays is similar, with some quirks.  
```python
arr[0]
```
Retruns the row in 0 index position.

```python
arr[0:3]
```
Returns a slice of the first three rows.

What about columns?  

First, get the row or rows you want, then column position like this:  

```python
arr[0:6,0] # first 6 rows before comma, first column after comma
```

So, array indexing is ```[rows,columns]``` or in other words ```[vertical axis, horizontal axis]```. Vertical axis = 0, Horizontal axis = 1. (typically)


```python
arr[0,0:] # first row, all columns
arr[1,0:3] # second row, columns 0, 1, 2
arr[0:3,1] # first three rows from column 2
arr[3,2] # value from fourth row, third column

A 3x3 slice of the top left corner:
```python
arr[0:3,0:3]
```

What will this do?  
```python
arr[0:3,0:3] + 100
```

```python
arr
```

Note that arrays are "mutable", meaning the values can be reassigned. Modify the same code:  
```python
arr[0:3,0:3] += 100
```

```python
arr
```

Modify the same code to return it to it's original form:

You can use various methods to broadcast to an array, such as ```np.ones()``` or ```np.zeros()```:

```python
np.ones((10,10)) # shape of the desired array is first parameter, which is provided in parentheses aka tuple
```

Of course, you can cast to the size of another array easily:  
```python
arr1 = np.ones((arr.shape))
```

You can do arithmetic operations on arrays. If you want to apply a formula to a whole array (array to scalar), it's very easy:  

```python
arr2 = arr1 * 2
```

It's also easy to do arithmetic operations on multiple arrays of the same shape:  

```python
arr * arr2
```

Note those values are now float type. Sometimes you may need to cast to a different data type, such as integer to float or vice versa:  
```python
arr3 = (arr * arr2).astype(int)
```

A useful method is ```np.where()```. This comes in handy in a variety of scenarios where you need to select based on conditions. 

First parameter is the condition, second parameter is the result if true, third parameter if the result is false. 

```python
np.where(arr>5,1,0)
```

Masking:

Create a random array:

This array will be populated by whole numbers between 1 and 100.

```python
np.random.randint(1,100,(10,10))
```

#### Multidimensional arrays  

You can stack arrays of the same size on top of one another for a multidimensional array.

```python
a1 = np.random.randint(1,100,(10,10))
a2 = np.random.randint(1,100,(10,10))
a3 = np.random.randint(1,100,(10,10))
```

```python
stack = np.stack([a1,a2,a3])
```

Or, a better way:  

```python
arrays = []
for n in range(3):
    arrays.append(np.random.randint(1,100,(10,10)))

stack = np.stack(arrays)
```

```python
stack.shape
```

Now, 0 axis is the multidimensional one.  

Generate a derived mean array from all three:  

```python
np.mean(stack, axis=0)

Or,
```python
stack.mean((0))
```

### Working with data arrays in practice.  

Here is a quick geospatial demonstration using a digital elevation model.  

I'll use the rasterio library to transform a geoTiff file into an array, run a processing operation over the array, and write a new output geoTiff.  

*Note*, you probably don't have rasterio installed, but you can always open your command line and type ```conda install rasterio```.  

```python
import rasterio as rio
```

In [ ]:
import rasterio as rio
import os
import matplotlib.pyplot as plt

In [ ]:
data_dir = r'C:\Users\phwh9568\Workshops\Python_Data_Camp\data'
file = 'Flatirons_DEM_1m.tif'

In [ ]:
dem = rio.open(os.path.join(data_dir, file))

In [ ]:
plt.imshow(dem.read(1))

Or:  
```python
from rasterio.plot import show
```

In [ ]:
from rasterio.plot import show

```python
show(dem)
```

Note matplot lib's colormaps: https://matplotlib.org/stable/tutorials/colors/colormaps.html

In [ ]:
show(dem, cmap='gray')

Plotting matplotlib style:
```python
fig, ax = plt.subplots(1,figsize=(12,12))
show(dem.read(1), transform=dem.transform, cmap='plasma', ax=ax)
```

In [ ]:
fig, ax = plt.subplots(1,figsize=(12,12))
show(dem.read(1), transform=dem.transform, cmap='plasma', ax=ax)

When working with geospatial raster data, knowing the coordinate system, size, and transformation information are all crucial for writing outputs:  

```python
dem.profile
```

Translate the DEM to a numpy array:  

```python
demArray = dem.read(1)
```

In [ ]:
demArray.size

In [ ]:
demArray.shape

Now that the DEM is an array, there are endless ways we could manipulate it. Let's perform a common geospatial operation, a smoothing filter based on the mean of a cells based the cell and it's 8 neighboring cells' values.  

There are several approaches to this including iterating through every cell (slow), takign the mean from slices of the array (fast), or using the SciPy library which already includes mean filter functions (easy). 

SciPy is frequently used alongside NumPy to extend it's functionality. A lot of things do-able in NumPy are made easier with SciPy's more advanced suite of algorithms.  

```python
import scipy.ndimage
```

In [ ]:
import scipy.ndimage

In [ ]:
demMean = scipy.ndimage.uniform_filter(demArray, size=3)

In [ ]:
demMean

Now, write it out as a GeoTiff:

In [ ]:
with rio.open('demMean.tif', 'w', **dem.profile) as demMean_out:
    demMean_out.write(demMean,1)